In [ ]:
from basenji import bed
import dataset_hic
from basenji import dataset
from basenji import plots
from basenji import seqnn
from basenji import trainer
from basenji import metrics
import tensorflow as tf
import h5py
import numpy as np
import seaborn as sns
import pandas as pd

In [ ]:
with_chr = "/data/basset_tfr/keep_chr1_downsampled5p"
wo_chr = "/data/basset_tfr/wo_chr1_downsampled5p/"

In [ ]:
data_path_5p_wo_chr = "/data/basset_tfr/wo_chr1_downsampled5p"
data_hic_5p_wo_chr = dataset_hic.SeqDataset(data_path_5p_wo_chr,
                              split_label='test',
                              batch_size=32,
                              shuffle_buffer=8192,
                              hic_length=25000,
                              cell_id=120,
                              mode='eval',
                              tfr_pattern=None)

data_path_5p_keep_chr = "/data/basset_tfr/keep_chr1_downsampled5p"
data_hic_5p_keep_chr = dataset_hic.SeqDataset(data_path_5p_keep_chr,
                              split_label='test',
                              batch_size=32,
                              shuffle_buffer=8192,
                              hic_length=25000,
                              cell_id=120,
                              mode='eval',
                              tfr_pattern=None)

In [ ]:
path = "/data/basset_tfr/full_downsampled10p/"
data_10 = dataset_hic.SeqDataset(path,
                              split_label='test',
                              batch_size=64,
                              shuffle_buffer=8192,
                              hic_length=25000,
                              cell_id=120,
                              mode='eval',
                              tfr_pattern=None)


In [ ]:
data_10.dataset

In [ ]:
targets_keep_chr = data_hic_5p_keep_chr.numpy(return_inputs=False)

In [ ]:
targets_keep_chr = data_hic_5p_keep_chr.numpy(return_inputs=False)

In [ ]:
targets_wo_chr = data_hic_5p_wo_chr.numpy(return_inputs=False)

In [ ]:
import json
params_file = "/config/params_basset.json"
with open(params_file) as params_open:
    params = json.load(params_open)
params_model = params['model']
params_train = params['train']

In [ ]:
# initialize model
seqnn_model = seqnn.SeqNN(params_model)

seqnn_model.restore("", 0, trunk=False )
seqnn_model.build_ensemble(False, [])

In [ ]:
### import layers

trained_model = "/data/basset_model/wo_chr1_5p//model_best.h5"
trained_model_load = tf.keras.models.load_model(trained_model, custom_objects={'StochasticReverseComplement': layers.StochasticReverseComplement(),  
                                                                  'SwitchReverse': layers.SwitchReverse(), 'StochasticShift': layers.StochasticShift}, compile=False)

In [ ]:
import layers

trained_model_10 = "/raid/dkokot/data/basset_model/full_downsampled10p_hic_proper_384/model_best.h5"
trained_model_load_10 = tf.keras.models.load_model(trained_model_10, 
                                                   custom_objects={'StochasticReverseComplement': layers.StochasticReverseComplement(), 
                                                                    'SwitchReverse': layers.SwitchReverse(), 
                                                                    'StochasticShift': layers.StochasticShift}
                                                                   , compile=False)

In [ ]:
preds_keep = trained_model_load.predict(getattr(data_hic_5p_keep_chr, 'dataset', None))

In [ ]:
preds_wo = trained_model_load.predict(getattr(data_hic_5p_wo_chr, 'dataset', None))

In [ ]:
test_targets_ti_keep, test_targets_ti_wo = targets_keep_chr[:, :, 120], targets_wo_chr[:, :, 120]
ds_indexes = np.arange(0, preds_keep.shape[1], 8)

test_targets_ti_flat_wo = test_targets_ti_wo[:, ds_indexes].flatten(
).astype('float32')
test_targets_ti_flat_keep = test_targets_ti_keep[:, ds_indexes].flatten(
).astype('float32')

exclude
test_preds_ti_flat_wo = preds_wo[:, ds_indexes, :].flatten().astype(
'float32')
test_preds_ti_flat_wo = preds_wo[:, ds_indexes, :].flatten().astype(
'float32')

# keep
test_preds_ti_flat_keep = preds_keep[:, ds_indexes, :].flatten().astype(
'float32')
test_preds_ti_flat_keep = preds_keep[:, ds_indexes, :].flatten().astype(
'float32')

In [ ]:
# call peaks
from scipy.stats import poisson
def ben_hoch(p_values):
  """ Convert the given p-values to q-values using Benjamini-Hochberg FDR. """
  m = len(p_values)

  # attach original indexes to p-values
  p_k = [(p_values[k], k) for k in range(m)]

  # sort by p-value
  p_k.sort()

  # compute q-value and attach original index to front
  k_q = [(p_k[i][1], p_k[i][0] * m // (i + 1)) for i in range(m)]

  # re-sort by original index
  k_q.sort()

  # drop original indexes
  q_values = [k_q[k][1] for k in range(m)]

  return q_values


In [ ]:
def call_peaks(target):
    test_targets_ti_lambda = np.mean(target)
    test_targets_pvals = 1 - poisson.cdf(
    np.round(target) - 1, mu=test_targets_ti_lambda)
    test_targets_qvals = np.array(ben_hoch(test_targets_pvals))
    test_targets_peaks = test_targets_qvals < 0.01
    test_targets_peaks_str = np.where(test_targets_peaks, 'Peak',
                                        'Background')
    return test_targets_peaks, test_targets_peaks_str

In [ ]:
test_targets_peaks_wo, test_targets_peaks_str_wo = call_peaks(test_targets_ti_flat_wo)

In [ ]:
test_targets_peaks_keep, test_targets_peaks_str_keep = call_peaks(test_targets_ti_flat_keep)

In [ ]:
len(test_targets_peaks_wo[np.where(test_targets_peaks_wo == 1)])

In [ ]:
len(test_targets_peaks_keep[np.where(test_targets_peaks_keep == 1)])

In [ ]:
preds_keep, _ = call_peaks(test_preds_ti_flat_keep)
preds_wo, _ = call_peaks(test_preds_ti_flat_wo)

In [ ]:
%matplotlib inline
import matplotlib
# matplotlib.use('PDF')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score

In [ ]:
test_preds_ti_flat_keep[np.where(test_preds_ti_flat_keep>0.5)]

In [ ]:
# ROC
plt.figure()
fpr, tpr, _ = roc_curve(test_targets_peaks_keep, test_preds_ti_flat_keep)
auroc = roc_auc_score(test_targets_peaks_keep, test_preds_ti_flat_keep)
plt.plot(
  [0, 1], [0, 1], c='black', linewidth=1, linestyle='--', alpha=0.7)
plt.plot(fpr, tpr, c='black')
ax = plt.gca()
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.text(
  0.99, 0.02, 'AUROC %.3f' % auroc,
  horizontalalignment='right')  # , fontsize=14)
ax.grid(True, linestyle=':')

In [ ]:
test_loss_hic, test_metric1_hic, test_metric2_hic = evaluate_model(trained_model_load, data_hic_5p_wo_chr, loss_label='bce')

In [ ]:
test_loss_keep, test_metric1_keep, test_metric2_keep = evaluate_model(trained_model_load, data_hic_5p_keep_chr, loss_label='bce')

In [ ]:
# evaluate
import os
loss_10, auroc_10, auprc10 = evaluate_model(trained_model_load_10, data_10, loss_label='bce')